# IESB
# Graduação em Ciência de Dados
## Disciplina - Mineração de Dados e Inteligência Computacional
## Avaliação - 04/05/2020

### Descrição
A base de dados "Home Equity" possui dados pessoas e informações de empréstimo de 5.960 empréstimos recentes. Para cada empréstimo existem 12 variáveis registradas.
A variável alvo (`BAD`) indica quando o cliente não pagou o empréstimo (valor 1), e quando ele honrou o compromisso (valor 0).

### Objetivo
Criar um modelo de ML usando RandomForest para prever se determinado cliente irá ou não honrar o empréstimo contraído.

O desempenho do modelo deve ser medido pela métrica de acurácia (`accuracy_score`).

In [ ]:
# Arquivo de dados
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

**Itens avaliados:**

1. Carregamento dos dados
2. Verificação para saber se os dados foram importados corretamente
3. Pré-processamento dos dados (preenchimento de valores em branco, transformação de texto em número)
4. Separar dados em treino e validação
5. Instanciar e treinar uma `RandomForest`
6. Avaliação da performance do modelo pela métrica de acurácia (`accuracy_score`).
7. Buscar a melhoria da acurácia mudando os parâmetros do modelo RandomForest.

In [ ]:
import pandas as pd
import numpy as np
# Carregar os dados
df = pd.read_csv("/kaggle/input/hmeq.csv")

In [ ]:
# Verificar os dados
df.shape

In [ ]:
# Informações dos dados
df.info()

In [ ]:
# Olhando os dados
df.head()

In [ ]:
# Olhando quantas variáveis tem por coluna
df.nunique()
# A variável BAD tem 2 

In [ ]:
# Soma dos valores faltantes 
df.isnull().sum().values.sum()

In [ ]:
# Os dados faltantes por coluna
df.isnull().sum()

In [ ]:
# Preencher os valores NaN com a classe que mais aparece na coluna REASON
# É uma variavel do tipo object
df.REASON.fillna('DebtCon', inplace=True)

In [ ]:
# Preencher os valores NaN com a classe que mais aparece na coluna JOB
# É uma variavel do tipo object
df.JOB.fillna('Other', inplace=True)

In [ ]:
# Os NaN serão substituidos pela média cada classe
df.fillna(0,inplace=True)

In [ ]:
# Verificando como está os dados depois da substituição
df.head()

In [ ]:
# Converter de object para int
for col in df.columns:
    if df[col].dtype == 'object':
        df[col] = df[col].astype('category').cat.codes

In [ ]:
df.info()

In [ ]:
# Separar os dados em treino e validação
from sklearn.model_selection import train_test_split

train, valid = train_test_split(df, test_size=0.2, random_state=42)

train.shape, valid.shape

In [ ]:
# Definindo as feats
feats = [c for c in df.columns if c not in ['BAD']]

In [ ]:
# Importar o modelo
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=100, random_state=42, min_samples_leaf=3)

In [ ]:
# Treinar o modelo
rf.fit(train[feats], train['BAD'])

In [ ]:
pd.Series(rf.feature_importances_ , index=feats).sort_values().plot.barh()

Eu poderia ter apenas completado as variáveis numéricas que estavam com valores faltando e ter deixado as duas variáveis do tipo *object* de lado na hora de definir as *feats*.

In [ ]:
# Previsão na base de validação
preds_val = rf.predict(valid[feats])
preds_val

In [ ]:
# Importar a métrica de acurácia
from sklearn.metrics import accuracy_score
accuracy_score(valid['BAD'], preds_val)

In [ ]:
# Previsão na base de treino
preds_train = rf.predict(train[feats])
preds_train

In [ ]:
accuracy_score(train['BAD'], preds_train)

A acurácia nos dados de treino foi maior que nos dados de validação. Acurácia diz de um modo geral como o modelo perfomou.

# Bônus

Após terminar o exercício e verificar que está cumprindo todos os itens a serem avaliados, você pode fazer os itens a seguir para ganhar pontos extra na prova:

**Bônus 1:**
    
Plote uma matriz de confusão nas previsões dos dados de validação.

In [ ]:
# Importar a bibilioteca que permite plotar gráficos com base nos modelos do scikit learn
import scikitplot as skplt

In [ ]:
# Plotando a matriz de confusão para os dados de validação
skplt.metrics.plot_confusion_matrix(valid['BAD'], preds_val)

**Bônus 2:**
    
Explique o que é `precisão` na matriz de confusão acima e dê 1 exemplo onde isso pode ser útil.

A* precisão* é a proporção de positivos classificados corretamente, ou seja, dos classificasdos como positivos
quantos realmente são positivos. No caso acima os positivos são o zeros, porque ele indica que a pesssoa realmente
pagou o empréstimo. Isso é bom pois não precisaria se preocupar com pessoa que realmente vai pagar,e sim com
aquelas pessoas que o modelo previu que iria pagar e acabou não pagando e com as pessoas que ele previu que não
iria pagar mas acabou pagando. Nesse último seria ruim para quem precisa do empréstimo porque essa pessoa ficaria
sempre ligada naqueles em que ela acha que não vai pagar, porém essa insistencia pode atrapalhar o pagamento.

**Bônus 3:**
    
Explique o que é `recall` na matriz de confusão acima e dê 1 exemplo onde isso pode ser útil.

O *recall* nos diz o quão bom meu modelo é para prever os positivos.Na matriz acima o recall é visto no quadrado onde o True label e Predicted label são iguais a 1, onde está o númeor 190. O *recall* pode ser útil em casos que deve saber se um paciente está ou não doente.